In [ ]:

#create df from Admissions
import pandas as pd
#display many rows
pd.options.display.max_rows = 200 

import numpy as np

df = pd.read_csv("ADMISSIONS.csv.gz")
df.head()

# Explore/Summary

In [ ]:
df.describe()

In [ ]:
df.describe(include=["object"])

In [ ]:
df["HOSPITAL_EXPIRE_FLAG"].value_counts()

In [ ]:
df["DIAGNOSIS"].value_counts().head(40)

In [ ]:
df["DIAGNOSIS"].count()

In [ ]:
df["DIAGNOSIS"].nunique()

In [ ]:
df["DIAGNOSIS"].size

In [ ]:
df["HADM_ID"].nunique()

# Begin Visualization

In [ ]:
#Histogram
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots()
ax.hist(df['ETHNICITY'],  color='red', alpha=.3, edgecolor='black', bins=10)
ax.set(xlabel="Age")

plt.show()

In [ ]:

df["ETHNICITY"].value_counts().head(50)

In [ ]:
#Histogram
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots()
ax.hist(df['INSURANCE'],  color='red', alpha=.3, edgecolor='black', bins=10)
ax.set(xlabel="Ins Type")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
df_lived = df.loc[(df["HOSPITAL_EXPIRE_FLAG"] == 0)]
df_died = df.loc[(df["HOSPITAL_EXPIRE_FLAG"] == 1)]
ax.hist(df_died["INSURANCE"],  color='red', alpha=.3, edgecolor='black')
ax.set(xlabel="Ins_type", title="Not Survived")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
df_lived = df.loc[(df["HOSPITAL_EXPIRE_FLAG"] == 0)]
df_died = df.loc[(df["HOSPITAL_EXPIRE_FLAG"] == 1)]

ax.hist(df_lived["INSURANCE"],  color='red', alpha=.3, edgecolor='black')
ax.set(xlabel="Ins_type", title="Survived")
plt.show()

In [ ]:

#Not helpful but good code for the future
import warnings
warnings.filterwarnings('ignore')

g = sns.catplot("HOSPITAL_EXPIRE_FLAG", col="INSURANCE", col_wrap=4,
                data=df,
                kind="count", height=2.5, aspect=.8)

# Group Ethnicities

In [ ]:
#Group Ethnicitie
df["ETHNICITY"] = df['ETHNICITY'].replace(['PORTUGESE', 'WHITE - OTHER EUROPEAN','WHITE', 'WHITE - EASTERN EUROPEAN', 'WHITE - RUSSIAN', 'WHITE - BRAZILIAN'],"White")
df["ETHNICITY"] = df['ETHNICITY'].replace(['BLACK/AFRICAN AMERICAN', 'BLACK/CAPE VERDEAN', 'BLACK/HAITIAN', 'CARIBBEAN ISLAND', 'BLACK/AFRICAN'],"Black")
df["ETHNICITY"] = df['ETHNICITY'].replace(['HISPANIC/LATINO - HONDURAN', 'HISPANIC/LATINO - MEXICAN', 'HISPANIC/LATINO - COLOMBIAN', 'HISPANIC/LATINO - SALVADORAN', 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)','SOUTH AMERICAN','HISPANIC/LATINO - GUATEMALAN', 'HISPANIC/LATINO - PUERTO RICAN', 'HISPANIC OR LATINO', 'HISPANIC/LATINO - CUBAN','HISPANIC/LATINO - DOMINICAN'],"Latino")
df["ETHNICITY"] = df['ETHNICITY'].replace(["OTHER", 'UNKNOWN/NOT SPECIFIED', 'PATIENT DECLINED TO ANSWER', 'MULTI RACE ETHNICITY', 'UNABLE TO OBTAIN'],"Other")
df["ETHNICITY"] = df['ETHNICITY'].replace(['AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE', 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',  'AMERICAN INDIAN/ALASKA NATIVE' ], "Indigenous")
df["ETHNICITY"] = df['ETHNICITY'].replace(['ASIAN - JAPANESE', 'ASIAN - THAI', 'ASIAN - VIETNAMESE','ASIAN - ASIAN INDIAN','ASIAN - CAMBODIAN','ASIAN - KOREAN', "ASIAN", 'ASIAN - CHINESE', 'ASIAN - OTHER', 'ASIAN - FILIPINO'],"Asian")
df["ETHNICITY"] = df['ETHNICITY'].replace('MIDDLE EASTERN',"Mid-East")

In [ ]:

df["ETHNICITY"].value_counts()

In [ ]:
df["ETHNICITY"] = df['ETHNICITY'].replace(['PORTUGUESE'],"White")
df["ETHNICITY"].value_counts()

In [ ]:
df.columns

# Limit df to heart disease

In [ ]:
#(df["DIAGNOSIS"]=='DYSPNEA')|
#(df["DIAGNOSIS"]=='SHORTNESS OF BREATH')|
#(df["DIAGNOSIS"]=='CHEST PAIN')|


df_hrt_dz = df.loc[(df["DIAGNOSIS"]=='CONGESTIVE HEART FAILURE')|(df["DIAGNOSIS"]=='CORONARY ARTERY DISEASE')| 
                   (df["DIAGNOSIS"]=='CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS GRAFT /SDA')|
                   (df["DIAGNOSIS"]=='CORONARY ARTERY DISEASE\CATH')|(df["DIAGNOSIS"]=='ACUTE CORONARY SYNDROME')|
                   (df["DIAGNOSIS"]=='MYOCARDIAL INFARCTION'),
                   ['SUBJECT_ID', 'HADM_ID', 'MARITAL_STATUS', 'ETHNICITY', 'DIAGNOSIS',  'HOSPITAL_EXPIRE_FLAG', 'ADMISSION_TYPE', 'ADMITTIME', 'ADMISSION_LOCATION']]
df_hrt_dz["HOSPITAL_EXPIRE_FLAG"].value_counts()

# Explore new df

In [ ]:
fig, ax = plt.subplots()
ax.hist(df_hrt_dz['DIAGNOSIS'],  color='red', alpha=.3, edgecolor='black', bins=10)
ax.set(xlabel="Dz/Symptom")

plt.show()

In [ ]:
df_hrt_dz[['DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG']].groupby(['DIAGNOSIS']).mean().sort_values(by='DIAGNOSIS', ascending=False)

# Combine Admission w/ pt table to get age, gender

In [ ]:
df_pt = pd.read_csv("PATIENTS.csv.gz")
df_hrt_dz = df_hrt_dz.merge(df_pt, on='SUBJECT_ID', how='left')
df_hrt_dz.head()

In [ ]:
type(df_hrt_dz['DOB'])

In [ ]:
df_hrt_dz["ADMITTIME"] = pd.to_datetime(df_hrt_dz["ADMITTIME"]).dt.date
df_hrt_dz["DOB"] = pd.to_datetime(df_hrt_dz["DOB"]).dt.date

df_hrt_dz['Age'] = df_hrt_dz.apply(lambda e: (e['ADMITTIME'] - e['DOB']).days/365, axis=1)

#df_hrt_dz['Age'] = ((df_hrt_dz["ADMITTIME"]- df_hrt_dz['DOB']).dt.days) //365

df_hrt_dz['Age'].head()

In [ ]:
df_hrt_dz = df_hrt_dz[df_hrt_dz['Age'] < 90]
df_hrt_dz['Age'].head()

In [ ]:
df_hrt_dz.loc[(df_hrt_dz["HOSPITAL_EXPIRE_FLAG"] == 0)].mean()

In [ ]:
df_hrt_dz.loc[(df_hrt_dz["HOSPITAL_EXPIRE_FLAG"] == 1, "Age")].mean()

In [ ]:
df_hrt_dz.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
df_hrt_lived = df_hrt_dz.loc[(df_hrt_dz["HOSPITAL_EXPIRE_FLAG"] == 0)]
df_hrt_died = df_hrt_dz.loc[(df_hrt_dz["HOSPITAL_EXPIRE_FLAG"] == 1)]
ax.hist(df_hrt_lived["ADMISSION_TYPE"],  color='red', alpha=.3, edgecolor='black')
ax.set(xlabel="admin_type", title="Survived")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
df_hrt_lived = df_hrt_dz.loc[(df_hrt_dz["HOSPITAL_EXPIRE_FLAG"] == 0)]
df_hrt_died = df_hrt_dz.loc[(df_hrt_dz["HOSPITAL_EXPIRE_FLAG"] == 1)]
ax.hist(df_hrt_died["ADMISSION_TYPE"],  color='red', alpha=.3, edgecolor='black')
ax.set(xlabel="admin_type", title="Not Survived")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
df_hrt_lived = df_hrt_dz.loc[(df_hrt_dz["HOSPITAL_EXPIRE_FLAG"] == 0)]
df_hrt_died = df_hrt_dz.loc[(df_hrt_dz["HOSPITAL_EXPIRE_FLAG"] == 1)]
ax.hist(df_hrt_lived["ADMISSION_LOCATION"],  color='red', alpha=.3, edgecolor='black')
ax.set(xlabel="admin_loc", title="Survived")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
df_hrt_lived = df_hrt_dz.loc[(df_hrt_dz["HOSPITAL_EXPIRE_FLAG"] == 0)]
df_hrt_died = df_hrt_dz.loc[(df_hrt_dz["HOSPITAL_EXPIRE_FLAG"] == 1)]
ax.hist(df_hrt_died["ADMISSION_LOCATION"],  color='red', alpha=.3, edgecolor='black')
ax.set(xlabel="admin_loc", title="Not Survived")

plt.show()

# Drop some columns

In [ ]:
df_hrt_dz.drop(["DOD", 'DOD_HOSP', 'DOD_SSN', 'EXPIRE_FLAG', "ADMISSION_LOCATION"], axis=1, inplace=True)
df_hrt_dz.head()

In [ ]:
df_hrt_dz["GENDER"].unique()

# Binary Encode Gender

In [ ]:
df_hrt_dz["GENDER"] = df_hrt_dz["GENDER"].map({'M':1, 'F':0})
df_hrt_dz.head()

# One hot Coding

In [ ]:
df_hrt_dz = pd.get_dummies(df_hrt_dz, columns=["ADMISSION_TYPE", "ETHNICITY", "MARITAL_STATUS"])
df_hrt_dz.columns

In [ ]:
df_hrt_dz.shape

In [ ]:
df_hrt_dz.describe()

In [ ]:

df_hrt_dz["HADM_ID"].nunique()